In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
from sklearn import metrics
import os, sys

from phm08ds.models import experiment

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual


## Load Dataset

In [47]:
folderpath = '../../data/interim/'
data_completed = pd.read_csv(folderpath + 'data_preprocessed.csv')
data_completed.head()

,unit,time_step,operational_setting_1,operational_setting_2,operational_setting_3,Sensor_0,Sensor_1,Sensor_2,Sensor_3,Sensor_4,...,Sensor_13,Sensor_14,Sensor_15,Sensor_16,Sensor_17,Sensor_18,Sensor_19,Sensor_20,Health_state,Operational_condition
0,1,1,10.0047,0.2501,20.0,489.05,604.13,1499.45,1309.95,10.52,...,8120.83,8.6216,0.03,368,2319,100.0,28.58,17.1735,1,2
1,1,2,0.0015,0.0003,100.0,518.67,642.13,1584.55,1403.96,14.62,...,8132.87,8.3907,0.03,391,2388,100.0,38.99,23.3619,1,3
2,1,3,34.9986,0.8401,60.0,449.44,555.42,1368.17,1122.49,5.48,...,8063.84,9.3557,0.02,334,2223,100.0,14.83,8.8555,1,1
3,1,4,20.0031,0.7005,0.0,491.19,607.03,1488.44,1249.18,9.35,...,8052.30,9.2231,0.02,364,2324,100.0,24.42,14.7832,1,6
4,1,5,42.0041,0.8405,40.0,445.00,549.52,1354.48,1124.32,3.91,...,8083.67,9.2986,0.02,330,2212,100.0,10.99,6.4025,1,4


In [3]:
data_unlabel = data_completed.drop(labels=['Health_state', 'Operational_condition'], axis=1)

## Data preprocessing

Use the pipeline and mlp

In [37]:
from phm08ds.data.preprocessing import OperationalCondition

data_unlabel = data_completed.drop(labels=['Health_state', 'Operational_condition'], axis=1)

tf_op_cond = OperationalCondition()
op_cond = tf_op_cond.fit_transform(data_unlabel.loc[0])

from phm08ds.features.feature_selection import RemoveSensor

tf_select_sensor = RemoveSensor(sensors=[0,4,5,7,9,15,16,17,18])

data_almost = tf_select_sensor.fit_transform(data_unlabel).iloc[:,5:]

from sklearn.preprocessing import StandardScaler

data_mlp = tf_std = StandardScaler().fit_transform(data_almost)

[[10.0047  0.2501 20.    ]]


In [44]:
mlp_clf.predict_proba(data_mlp)[:,-1]

array([0.23958697, 0.35386553, 0.00982346, ..., 0.01743402, 0.01380116,
       0.00730301])

In [1]:
from sklearn.externals import joblib

classifiers = joblib.load('classifiers.pkl')
mlp_clf = classifiers['MLP']

def plot_series(unit, sensor):
    data_unit_sensor_buffer = data_completed.loc[data_completed['unit'] == unit,:]
    
    from phm08ds.data.preprocessing import OperationalCondition
        
    data_unlabel = data_unit_sensor_buffer.drop(labels=['Health_state', 'Operational_condition'], axis=1)
    
#     tf_op_cond = OperationalCondition()
#     op_cond = tf_op_cond.fit_transform(data_unlabel.loc[0])

    from phm08ds.features.feature_selection import RemoveSensor

    tf_select_sensor = RemoveSensor(sensors=[0,4,5,7,9,15,16,17,18])

    data_almost = tf_select_sensor.fit_transform(data_unlabel).iloc[:,5:]

    from sklearn.preprocessing import StandardScaler

    data_mlp = StandardScaler().fit_transform(data_almost)
        
    plt.figure(figsize=(30*0.39, 10*0.39))    
    sns.lineplot(x='time_step', y='Sensor_' + str(sensor), data=data_unit_sensor_buffer, hue='Health_state', palette='Wistia')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    
    print(mlp_clf.predict_proba(data_mlp[220,:].reshape(1,-1))[0,2])
    
    
    
interact(plot_series, unit=(1,100,1), sensor=(1,20,1))

NameError: name 'interact' is not defined

Get sensors that a I like it better

Before feeding to the classifier, let's remove unwanted information, such as unit, time_step and operational settings.

In [ ]:
from phm08ds.features.feature_selection import RemoveInfo

tf_remove_info = RemoveInfo()

data_with_features = tf_remove_info.fit_transform(data_op_1)
data_with_features.head()

## Create a new feature set based on HOS

We need to normalize our data. Let's use Z-score standardization.

In [ ]:
from sklearn.preprocessing import StandardScaler

tf_std_scaller = preprocessing.StandardScaler()
data_with_features_std = tf_std_scaller.fit_transform(data_with_features.drop(labels='Health_state', axis=1))
data_with_features_std

In [ ]:
labels_op_1 = np.array(data_with_features['Health_state'])
labels_op_1

# Classification steps

## Load Experiment model

In [ ]:
from phm08ds.models import experiment

## Define classifiers and its specifications

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.externals import joblib

In [ ]:
mlp_clf = MLPClassifier(hidden_layer_sizes=(50,), activation='tanh', solver='adam', tol=1e-8, warm_start=True)

## Put all clf in a dictionary:

In [ ]:
classifiers = {'MLP': mlp_clf}

## Train Classifiers and test them

Stratified cross-validation for model selection are going to be used.

In [ ]:
kfolds = 10
clf_outputs = experiment.run_classifiers(data_with_features_std, labels_op_1, classifiers, kfolds)

## Performance assessment

Saving variables in a dictionary:

In [ ]:
results = {}
results['train'] = experiment.results_clf(4, clf_outputs['train']['true'], clf_outputs['train']['pred'])
results['test'] = experiment.results_clf(4, clf_outputs['test']['true'], clf_outputs['test']['pred'])

## Savel results, models and pipeline to a .pkl file 

In [ ]:
from sklearn.pipeline import Pipeline

data_preprocessing = Pipeline([('select_sensors', tf_select_sensors),
                               ('remove_info', tf_remove_info),
                               ('std_scaler', tf_std_scaller)
                              ])

In [ ]:
experiment.save_models(classifiers)
experiment.save_pipeline(data_preprocessing)

## Save results to CSVs and figures

In [ ]:
experiment.export_results(results['test'], 'test')
experiment.export_results(results['train'], 'train')